# Parametric Curve Fitting Analysis

This notebook demonstrates the process of estimating unknown parameters θ, M, X in a parametric curve model.

In [ ]:
import sys
sys.path.insert(0, '../src')

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from data_loader import load_data, get_uniform_t_values
from model import predict, deg_to_rad
from optimizer import fit_params
from plotting import plot_fit, plot_residuals
from utils import save_params_json

%matplotlib inline

## 1. Load Data

Load the observed (x,y) data points from CSV file.

In [ ]:
# Load data
df = load_data('../data/xy_data.csv')
print(f"Loaded {len(df)} data points")
df.head()

## 2. Visualize Raw Data

In [ ]:
plt.figure(figsize=(10, 8))
plt.scatter(df['x'], df['y'], alpha=0.5, s=20)
plt.xlabel('x')
plt.ylabel('y')
plt.title('Observed Data Points')
plt.grid(True, alpha=0.3)
plt.show()

## 3. Parametric Model

The parametric equations are:

$$x(t) = t \cos(\theta) - e^{M|t|} \sin(0.3t) \sin(\theta) + X$$

$$y(t) = 42 + t \sin(\theta) + e^{M|t|} \sin(0.3t) \cos(\theta)$$

Where:
- $0^\circ < \theta < 50^\circ$
- $-0.05 < M < 0.05$
- $0 < X < 100$
- $6 < t < 60$

## 4. Run Optimization

In [ ]:
# Run optimization
result = fit_params(df, n_samples=200, n_restarts=10, seed=42)

print("\nOptimization Results:")
print(f"θ = {result['theta_deg']:.4f}° ({result['theta_rad']:.6f} rad)")
print(f"M = {result['M']:.6f}")
print(f"X = {result['X']:.4f}")
print(f"L1 Score = {result['l1']:.6f}")

## 5. Visualize Fitted Curve

In [ ]:
# Generate fitted curve
t_curve = get_uniform_t_values(500)
x_curve, y_curve = predict(t_curve, result['theta_deg'], result['M'], result['X'])

# Plot
plt.figure(figsize=(10, 8))
plt.scatter(df['x'], df['y'], alpha=0.5, s=20, label='Observed Data')
plt.plot(x_curve, y_curve, 'r-', linewidth=2, label='Fitted Curve', alpha=0.8)
plt.xlabel('x')
plt.ylabel('y')
plt.title('Parametric Curve Fit')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

## 6. Analyze Residuals

In [ ]:
# Compute residuals
t_samples = get_uniform_t_values(len(df))
x_pred, y_pred = predict(t_samples, result['theta_deg'], result['M'], result['X'])

residuals = np.sqrt((x_pred - df['x'].values)**2 + (y_pred - df['y'].values)**2)

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

ax1.plot(residuals, 'o-', alpha=0.6, markersize=4)
ax1.set_xlabel('Data Point Index')
ax1.set_ylabel('Residual Distance')
ax1.set_title('Residuals vs Data Point')
ax1.grid(True, alpha=0.3)

ax2.hist(residuals, bins=30, alpha=0.7, edgecolor='black')
ax2.set_xlabel('Residual Distance')
ax2.set_ylabel('Frequency')
ax2.set_title('Residual Distribution')
ax2.axvline(np.mean(residuals), color='r', linestyle='--', label=f'Mean: {np.mean(residuals):.4f}')
ax2.legend()
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 7. Save Results

In [ ]:
# Save parameters
save_params_json(result, '../results/params.json')
print("Parameters saved to results/params.json")

# Save plots
plot_fit(df, result, output_path='../results/fit_plot.png')
plot_residuals(df, result, output_path='../results/residuals_plot.png')

## 8. Final Equation

The fitted parametric equations with estimated parameters are:

$$x(t) = t \cos(\theta) - e^{M|t|} \sin(0.3t) \sin(\theta) + X$$

$$y(t) = 42 + t \sin(\theta) + e^{M|t|} \sin(0.3t) \cos(\theta)$$

Where the estimated parameters are shown in the cell above.